## Пример запуска персонального AI-ассистента с A*-алгоритмом поиска

In [ ]:
import sys
# TO CHANGE
BASEDIR = "../../.."
sys.path.insert(0, BASEDIR)

In [ ]:
from src import PersonalAI, PersonalAIConfig, QAPipelineConfig, MemPipelineConfig, \
      GraphModelConfig, EmbeddingsModelConfig, EmbedderModelConfig

from src.db_drivers import KeyValueDriverConfig, GraphDriverConfig, VectorDriverConfig
from src.db_drivers.kv_driver import DEFAULT_INMEMORYKV_CONFIG
from src.db_drivers.graph_driver import DEFAULT_INMEMORYGRAPH_CONFIG
from src.db_drivers.vector_driver import VectorDBConnectionConfig

from src.pipelines.qa.knowledge_retriever import AStarGraphSearchConfig, AStarMetricsConfig
from src.pipelines.qa import QueryLLMParserConfig, KnowledgeComparatorConfig, KnowledgeRetrieverConfig, QALLMGeneratorConfig

from src.pipelines.memorize import LLMExtractorConfig, LLMUpdatorConfig

from src.utils import NodeType, Logger


### 1. Задаём конфигурацию графа знаний

Для создания объекта класса PersonalAI (верхнеуровневый класс персонального ассистента) необходимо выполнить конфигурацию трёх основных его компонент:
* Графа знаний, который будет выступать в роли памяти ассистента и хранить поступающую информацию.
* QA-конвейера, с помощью которого ассистент будет отвечать на user-вопросы.
* Memorize-конвейера, с помощью которого ассистент будет добавлять новую информацию в память и выполнять её актуализацию.

Граф знаний состоит из двух моделей хранения информации: векторной и графовой. Соответственно нам потребуется инициализировать два конфигурационных data-класса: `GraphModelConfig` и `EmbeddingModelConfig`.

В конфиге для графовой модели потребуется указать параметры подключения к одной из доступных графовых бд. В нашем случае мы подключаемся к самописной моделе, которая хранить граф в оперативной памяти.

In [3]:
# Graph model configuration
GRAPH_STORAGE_CONFIG = GraphDriverConfig(db_vendor='inmemory_graph', db_config=DEFAULT_INMEMORYGRAPH_CONFIG)
GRAPH_MODEL_CONFIG = GraphModelConfig(driver_config=GRAPH_STORAGE_CONFIG)

В конфиге для векторной модели потребуется отдельно указать параметры подключения к двум векторным бд, где будут храниться эмбеддинги триплетов и вершин из графовой модели соответственно.

In [4]:
# Vector model configuration
NODES_DB_PATH = f'{BASEDIR}/data/graph_structures/vectorized_nodes/testing'
TRIPLETS_DB_PATH = f'{BASEDIR}/data/graph_structures/vectorized_triplets/testing'
NEED_TO_CLEAR = True

VECTOR_NODES_STORAGE_CONFIG = VectorDriverConfig(db_config=VectorDBConnectionConfig(path=NODES_DB_PATH, need_to_clear=NEED_TO_CLEAR))
VECTOR_TRIPLETS_STIRAGE_CONFIG = VectorDriverConfig(db_config=VectorDBConnectionConfig(path=TRIPLETS_DB_PATH, need_to_clear=NEED_TO_CLEAR))

Также потребуется указать параметры для инициализации класса, с помощью которого будет выполняться перевод текста в его векторное представления (полечение эмбеддингов).

In [5]:
DEVICE = 'cuda'
EMBEDDER_MODEL_PATH = f'{BASEDIR}/models/intfloat/multilingual-e5-small'
EMBEDDER_MODEL_CONFIG = EmbedderModelConfig(model_name_or_path=EMBEDDER_MODEL_PATH, device=DEVICE)

In [6]:
VECTOR_MODEL_CONFIG = EmbeddingsModelConfig(
    nodesdb_driver_config=VECTOR_NODES_STORAGE_CONFIG,
    tripletsdb_driver_config=VECTOR_TRIPLETS_STIRAGE_CONFIG,
    embedder_config=EMBEDDER_MODEL_CONFIG)

Далее нам потребуется сконфигурировать QA- и Memorize-пайплайны.

В рамках QA-конфигурации мы задаём алгоритм поиска, с помощью которого будет извлекаться информации из графа знаний. В нашем случае это будет A*-алгоритм. Для этого нам потребуется указать его идентификатор (в виде ключевого слова) и соответствующий ему data-класс `AStarGraphSearchConfig` с заданными гиперпараметрами.

In [7]:
RETRIEVER_NAME = 'astar'

KV_STORAGE_CONFIG = KeyValueDriverConfig(db_vendor='inmemory_kv', db_config=DEFAULT_INMEMORYKV_CONFIG)

RETRIEVER_HYPERP = AStarGraphSearchConfig(
    metrics_config=AStarMetricsConfig(h_metric_name='ip',kvdriver_config=KV_STORAGE_CONFIG),
    max_depth=-1, max_passed_nodes=-1,
    accepted_node_types=[NodeType.object , NodeType.hyper, NodeType.episodic]
)

Для QueryParser- и QAGenerator- стадий мы указываем язык, который будет использоваться для генерации необходимой информации c помощью LLM-агента. В рамках библиотеки прддерживается два языка: русский и английский. Мы будем использовать английский.

In [8]:
LANGUAGE = 'en'

In [9]:
# QA-pipeline configuration
QA_PIPELINE_CONFIG = QAPipelineConfig(
    query_parser_config=QueryLLMParserConfig(lang=LANGUAGE),
    knowledge_comparator_config=KnowledgeComparatorConfig(),
    knowledge_retriever_config=KnowledgeRetrieverConfig(
        retriever_method=RETRIEVER_NAME,retriever_config=RETRIEVER_HYPERP),
    answer_generator_config=QALLMGeneratorConfig(lang=LANGUAGE))

В рамках Memorize-конфигурации мы задаём только язык обрабатываемого текста, чтобы на шаге запуска LLM-агента выбрать болен оптимальные промпты и функцию парсинга ответа.

In [10]:
# Memorize-pipeline configuration
DELETE_OBSOLETE_INFO = True

MEM_PIPELINE_CONFIG = MemPipelineConfig(
    extractor_config=LLMExtractorConfig(lang=LANGUAGE),
    updator_config=LLMUpdatorConfig(lang=LANGUAGE, delete_obsolete_info=DELETE_OBSOLETE_INFO))

Таким образом, у нас была получена конфигурация гиперпараметров для инициализации персонального ассистена. 

In [11]:
PERSONALAI_CONFIG = PersonalAIConfig(
    graph_struct_config=GRAPH_MODEL_CONFIG,
    embedds_struct_config=VECTOR_MODEL_CONFIG,
    qa_pipeline_config=QA_PIPELINE_CONFIG,
    mem_pipeline_config=MEM_PIPELINE_CONFIG,
    log=Logger('log/main'))

#### 2. Инициализируем персонального ассистента

In [ ]:
personalai = PersonalAI(config=PERSONALAI_CONFIG)

#### 3. Пример работы Memorize-конвейера

Подготавливаем набор текстов на естественном языке (в нашем случае на английском) для их последующего сохранения в память ассистента.

In [13]:
messages = [
    "Mikhail Menshchikov is currently a second-year master's student at ITMO.",
    "Mikhail Menshchikov is studying in the Master's program 'Deep Learning and Generative AI'",
    "Mikhail Menshchikov completed his bachelor's degree at Petrozavodsk State University",
    "Petrozavodsk State University is where Mikhail Menshchikov received his bachelor's degree.",
    "Mikhail Menshchikov studied at Petrozavodsk State University and received a bachelor's degree."]
properties = [dict() for _ in range(len(messages))]

С помощью метода update_memory сохраняем сформированный набор информации в память.

In [14]:
for text, prop in zip(messages, properties):
    _, info = personalai.update_memory(text, prop)

#### 4. Пример работы QA-конвейера

Теперь мы можем задавать вопросы ассистенту по имеющимся у него знаниях в памяти и получать ответы.

In [15]:
answer, info = personalai.answer_question("What program is Mikhail Menshchikov studying for his master's degree?")
print(answer)

Chain of thought: Mikhail Menshchikov is pursuing a master's degree, and we need to determine the specific program he is enrolled in. According to the information provided, Mikhail Menshchikov is studying in the master's program called "deep learning and generative ai."

Final answer: deep learning and generative ai


In [16]:
answer, info = personalai.answer_question("Where did Mikhail Menshchikov receive his bachelor's degree?")
print(answer)

Chain of thought: The information provided does not specify where Mikhail Menshchikov received his bachelor's degree.
Final answer: The information provided does not allow us to determine the specific institution where Mikhail Menshchikov received his bachelor's degree.
